In [93]:
import json
import pandas as pd
import argparse
import torch
import difflib
import nltk
import regex as re
import numpy as np
import MeCab
import pickle

from tqdm import tqdm
from collections import defaultdict
from transformers import AutoModelForMaskedLM, AutoTokenizer
from datasets import load_dataset_builder, load_dataset, get_dataset_split_names, get_dataset_config_names

In [111]:
# dataset_name = "Paul/hatecheck"

dataset_name = "Paul/hatecheck-spanish"
# dataset_name = "piuba-bigdata/contextualized_hate_speech"

# dataset_name = "Paul/hatecheck-portuguese"
# dataset_name = "hate_speech_portuguese"
# dataset_name = "Paul/hatecheck-german"
# dataset_name = "Paul/hatecheck-italian"
# dataset_name = "flax-sentence-embeddings/Gender_Bias_Evaluation_Set"

# dataset_name = "Paul/hatecheck-arabic"

# dataset_name = "Paul/hatecheck-french"

# dataset_name = "Paul/hatecheck-mandarin"

# dataset_name = "Paul/hatecheck-dutch"
# dataset_name = "Paul/hatecheck-polish"
# dataset_name = "Paul/hatecheck-hindi"

lang = 'es'

In [112]:
dataset = load_dataset(dataset_name, split="test")  #, split="test"
dataset[0]

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1817.29it/s]
Generating test split: 0 examples [00:00, ? examples/s]/home/cass/anaconda3/envs/eva/lib/python3.11/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)
                                                       

Dataset csv downloaded and prepared to /home/cass/.cache/huggingface/datasets/Paul___csv/Paul--hatecheck-spanish-8297884bca10d574/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


{'mhc_case_id': 'spanish-1',
 'functionality': 'derog_neg_emote_h',
 'test_case': 'Odio a los negros.',
 'label_gold': 'hateful',
 'target_ident': 'black people',
 'ref_case_id': None,
 'ref_templ_id': None,
 'templ_id': 1,
 'case_templ': None,
 'gender_male': 'Odio a los [male_IDENT_P].',
 'gender_female': 'Odio a las [female_IDENT_P].',
 'label_annotated': "['hateful', 'hateful', 'hateful']",
 'label_annotated_maj': 'hateful',
 'disagreement_in_case': False,
 'disagreement_in_template': False}

In [113]:
idt_list = []
sentence_example = []

for i in dataset:
    if i['label_gold'] == 'hateful':
        idt = i['target_ident'].split()
        for w in idt:
            if w not in idt_list:
                idt_list.append(w)
                sentence_example.append(i['test_case'])
       
        

idt_list.remove('people')
idt_list


['black', 'gay', 'trans', 'indigenous', 'jews', 'disabled', 'women']

In [114]:
sentence_example

['Odio a los negros.',
 'Odio a los negros.',
 'Odio a los gays.',
 'Odio a los trans.',
 'Odio a los indígenas.',
 'Odio a los judíos.',
 'Odio a los discapacitados.',
 'Odio a las mujeres.']

In [98]:
import logging
import sys

logging.basicConfig(filename=f'./hate/{lang}/identity_terms.txt',level=logging.info)

logging.info(sentence_example)
logging.info(idt_list)

In [99]:
idt_list = ['trans', 'gay', 'disabili', 'musulmani', 'immigrati', 'donne', 'neri', 'persone di colore']
logging.info(idt_list)

In [100]:
# df = pd.DataFrame(list(zip(idt_list, sentence_example)),
#                columns =['Identity terms', 'Sentences'])
# df.to_csv(f'./hate/{lang}/identity_terms.csv')

In [101]:
hate = []
nonhate = []
for i in dataset:
    if i['label_gold'] == 'hateful':
        hate.append(i['test_case'])
    else:
        nonhate.append(i['test_case'])


hate_idt = []
hate_nonidt = []
for i in hate:
    if len([each for each in idt_list if each.lower() in i.split()])>0: # hate speech with idt
        hate_idt.append(i)
    else:
        hate_nonidt.append(i) # hate without idt / non hate with idt


nonhate_idt = []
nonhate_nonidt = []
for i in nonhate:
    if len([each for each in idt_list if each.lower() in i.split()])>0: # hate speech with idt
        nonhate_idt.append(i)
    else:
        nonhate_nonidt.append(i) # hate without idt / non hate with idt

In [102]:
import json
with open(f"./hate/{lang}/hate_idt.json", 'w') as f:
    json.dump(hate_idt, f, indent=4, ensure_ascii=False) 

with open(f"./hate/{lang}/hate_nonidt.json", 'w') as f:
    json.dump(hate_nonidt, f, indent=4, ensure_ascii=False) 

with open(f"./hate/{lang}/nonhate_idt.json", 'w') as f:
    json.dump(nonhate_idt, f, indent=4, ensure_ascii=False) 

with open(f"./hate/{lang}/nonhate_nonidt.json", 'w') as f:
    json.dump(nonhate_nonidt, f, indent=4, ensure_ascii=False) 

In [103]:
# import json
# with open(f"./hateB/{lang}/hate_idt.json", 'w') as f:
#     json.dump(hate_idt, f, indent=4, ensure_ascii=False) 

# with open(f"./hateB/{lang}/hate_nonidt.json", 'w') as f:
#     json.dump(hate_nonidt, f, indent=4, ensure_ascii=False) 

# with open(f"./hateB/{lang}/nonhate_idt.json", 'w') as f:
#     json.dump(nonhate_idt, f, indent=4, ensure_ascii=False) 

# with open(f"./hateB/{lang}/nonhate_nonidt.json", 'w') as f:
#     json.dump(nonhate_nonidt, f, indent=4, ensure_ascii=False) 

In [104]:
# hate = []
# nonhate = []
# for i in dataset:
#     if i['label'] == 1:
#         hate.append(i['text'])
#     else:
#         nonhate.append(i['text'])


# hate_idt = []
# hate_nonidt = []
# for i in hate:
#     if len([each for each in idt_list if each.lower() in i.split()])>0: # hate speech with idt
#         hate_idt.append(i)
#     else:
#         hate_nonidt.append(i) # hate without idt / non hate with idt


# nonhate_idt = []
# nonhate_nonidt = []
# for i in nonhate:
#     if len([each for each in idt_list if each.lower() in i.split()])>0: # hate speech with idt
#         nonhate_idt.append(i)
#     else:
#         nonhate_nonidt.append(i) # hate without idt / non hate with idt

In [105]:
def get_idt(lang):
    if lang == 'de':
        idt_list = ["Muslime", "Juden", "Juden", "Weiße", "Islam", "Schwarze", "Muslime", "Frauen", "Weiße", "Schwule", "Schwarze", "Democat", "Islamisch", "Allah", "Jüdisch", "Lesbe", "Transgender", "Rasse", "Braun", "Frau", "Mexikaner", "Religion", "Homosexuell", "Homosexualität", "Afrikaner"]
        idt_list = idt_list + ["Frauen",  "Transmenschen", "Schwule",  "Schwarze",  "Behinderte","Juden","Flüchtlinge"]
        idt_list = list(set(idt_list))
    elif lang == 'en':
        idt_list = ['muslim','jew', 'jews', 'white', 'islam', 'blacks', 'muslims', 'women', 'whites', 
                    'gay', 'black', 'democat', 'islamic', 'allah', 'jewish','lesbian', 'transgender', 
                    'race', 'brown', 'woman', 'mexican', 'religion', 'homosexual', 'homosexuality', 'africans']
    elif lang == 'ja': 
        idt_list = ['イスラム教徒','ユダヤ人', 'ユダヤ人', '白人', 'イスラム教', '黒人', 'イスラム教徒', '女性', '白人', 'ゲイ', '黒人', 'デモキャット', 'イスラム教', 'アッラー', 'ユダヤ人', 'レズビアン', 'トランスジェンダー', '人種', '茶色', '女性', 'メキシコ人', '宗教', '同性愛者', '同性愛', 'アフリカ人']
    elif lang == 'es': 
        idt_list = ['Musulmán', 'judío', 'judíos', 'blanco', 'islam', 'negros', 'musulmanes', 'mujeres', 'blancos', 'gay', 'negro', 'democat', 'islámico', 'Alá', 'judío', 'lesbiana', 'transgénero', 'raza', 'marrón', 'mujer', 'mexicano', 'religión', 'homosexual', 'homosexualidad', 'africanos']
        
    elif lang == 'pt':
        idt_list = ['muçulmanos', 'judeu', 'judeus', 'brancos', 'islâmicos', 'negros', 'muçulmanos', 'mulheres', 'brancos', 'gay', 'negro', 'democat', 'islâmico', 'allah', 'judeu', 'lésbicas', 'transgénero', 'raça', 'pardo', 'mulher', 'mexicano', 'religião', 'homossexual', 'homossexualidade', 'africanos']
        idt_list = idt_list + ['muçulmano', 'judeu', 'judeus', 'branco', 'islã', 'negros', 'muçulmanos', 'mulheres', 'brancos', 'gay', 'negro', 'democrata', 'islâmico ', 'alá', 'judeu', 'lésbica', 'transgênero', 'raça', 'pardo', 'mulher', 'mexicano', 'religião', 'homossexual', 'homossexualidade', 'africanos']
        idt_list = list(set(idt_list))
        
    elif lang == 'fr': 
        idt_list = ['musulman', 'juif', 'juifs', 'blanc', 'islam', 'noirs', 'musulmans', 'femmes', 'blancs', 'gay', 'noir', 'démocat', 'islamique ', 'allah', 'juif', 'lesbienne', 'transgenre', 'race', 'marron', 'femme', 'mexicain', 'religion', 'homosexuel', 'homosexualité', 'africains']
    elif lang == 'du': 
        idt_list = ['moslim', 'jood', 'joden', 'blank', 'islam', 'zwarten', 'moslims', 'vrouwen', 'blanken', 'homo', 'zwart', 'demokat', 'islamitisch ', 'allah', 'joods', 'lesbisch', 'transgender', 'ras', 'bruin', 'vrouw', 'mexicaans', 'religie', 'homoseksueel', 'homoseksualiteit', 'afrikanen']
    elif lang == 'it': 
        idt_list = ['musulmano', 'ebreo', 'ebrei', 'bianco', 'islam', 'neri', 'musulmani', 'donne', 'bianchi', 'gay', 'neri', 'demogatti', 'islamici ', 'allah', 'ebrea', 'lesbica', 'transgender', 'razza', 'marrone', 'donna', 'messicana', 'religione', 'omosessuale', 'omosessualità', 'africani']
    elif lang == 'zh': 
        idt_list = ['穆斯林', '犹太', '犹太人', '白人', '伊斯兰', '黑人', '女', '白人', '工党', '真主', '犹太教', '拉拉', '变性', '种族', '棕色', '墨西哥', '宗教', '同性', '同性恋', '非洲人']
    elif lang == 'ar':
        idt_list = ["الله" ,"اليهود" ,"السحاقيات" ,"المتحولين جنسياً" ,"العرق" ,"البني" ,"المرأة" ,"المكسيكي" ,"الدين" ,"مثلي الجنس" ,"الشذوذ الجنسي" ,"الأفارقة"]
    elif lang == 'hi':
        idt_list = ['मुस्लिम', 'यहूदी', 'यहूदी', 'श्वेत', 'इस्लाम', 'अश्वेत', 'मुस्लिम', 'महिलाएं', 'गोरे', 'समलैंगिक', 'काला', 'डेमोकैट', 'इस्लामिक', 'अल्लाह', 'यहूदी', 'लेस्बियन', 'ट्रांसजेंडर', 'जाति', 'भूरा', 'महिला', 'मैक्सिकन', 'धर्म', 'समलैंगिक', 'समलैंगिकता', 'अफ्रीकी']
    
    return idt_list

idt_list = get_idt(lang)

In [106]:
def get_model_name(lang):
    if lang == 'de':
        model_name = 'deepset/gbert-base' 
    elif lang == 'de-xlm':
        model_name = 'xlm-roberta-large-finetuned-conll03-german'
    elif lang == 'en':
        model_name = 'bert-base-uncased'
    elif lang == 'en-xlm':
        model_name = 'xlm-mlm-ende-1024'
    elif lang == 'ja': 
        model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
    elif lang == 'ar': # Arabic
        model_name = 'aubmindlab/bert-base-arabertv02'
    elif lang == 'es': 
        model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
    elif lang == 'pt': 
        model_name = 'neuralmind/bert-base-portuguese-cased'
    elif lang == 'ru': 
        model_name = 'blinoff/roberta-base-russian-v0'
    elif lang == 'id':
        model_name = 'cahya/bert-base-indonesian-1.5G'
    elif lang == 'zh':
        model_name = 'hfl/chinese-bert-wwm-ext'
    elif lang == 'multi-xlm':
        model_name = 'xlm-mlm-100-1280'
    elif lang == 'multi-bert':
        model_name = 'bert-base-multilingual-uncased'
    return model_name


def get_idt(lang):
    if lang == 'de':
        idt_list = ["Muslime", "Juden", "Juden", "Weiße", "Islam", "Schwarze", "Muslime", "Frauen", "Weiße", "Schwule", "Schwarze", "Democat", "Islamisch", "Allah", "Jüdisch", "Lesbe", "Transgender", "Rasse", "Braun", "Frau", "Mexikaner", "Religion", "Homosexuell", "Homosexualität", "Afrikaner"]
    elif lang == 'en':
        idt_list = ['muslim','jew', 'jews', 'white', 'islam', 'blacks', 'muslims', 'women', 'whites', 'gay', 'black', 'democat', 'islamic', 'allah', 'jewish','lesbian', 'transgender', 'race', 'brown', 'woman', 'mexican', 'religion', 'homosexual', 'homosexuality', 'africans']
    elif lang == 'ja': 
        idt_list = ['イスラム教徒','ユダヤ人', 'ユダヤ人', '白人', 'イスラム教', '黒人', 'イスラム教徒', '女性', '白人', 'ゲイ', '黒人', 'デモキャット', 'イスラム教', 'アッラー', 'ユダヤ人', 'レズビアン', 'トランスジェンダー', '人種', '茶色', '女性', 'メキシコ人', '宗教', '同性愛者', '同性愛', 'アフリカ人']
    elif lang == 'es': 
        idt_list = ['Musulmán', 'judío', 'judíos', 'blanco', 'islam', 'negros', 'musulmanes', 'mujeres', 'blancos', 'gay', 'negro', 'democat', 'islámico', 'Alá', 'judío', 'lesbiana', 'transgénero', 'raza', 'marrón', 'mujer', 'mexicano', 'religión', 'homosexual', 'homosexualidad', 'africanos']
    elif lang == 'pt': 
        idt_list = ['muçulmanos', 'judeus', 'judeus', 'brancos', 'islâmicos', 'negros', 'muçulmanos', 'mulheres', 'brancos', 'gay', 'negro', 'democat', 'islâmico', 'allah', 'judeu', 'lésbicas', 'transgénero', 'raça', 'pardo', 'mulher', 'mexicano', 'religião', 'homossexual', 'homossexualidade', 'africanos']
    return idt_list

In [107]:
def calculate_aul(model, token_ids, log_softmax, attention):
    '''
    Given token ids of a sequence, return the averaged log probability of
    unmasked sequence (AULA or AUL).
    '''
    output = model(token_ids)
    logits = output.logits.squeeze(0)
    log_probs = log_softmax(logits)
    token_ids = token_ids.view(-1, 1).detach()
    token_log_probs = log_probs.gather(1, token_ids)[1:-1]
    if attention:
        attentions = torch.mean(torch.cat(output.attentions, 0), 0)
        averaged_attentions = torch.mean(attentions, 0)
        averaged_token_attentions = torch.mean(averaged_attentions, 0)
        token_log_probs = token_log_probs.squeeze(1) * averaged_token_attentions[1:-1]
    sentence_log_prob = torch.mean(token_log_probs)
    score = sentence_log_prob.item()

    hidden_states = output.hidden_states[-1][:,1:-1]
    hidden_state = torch.mean(hidden_states, 1).detach().cpu().numpy()

    return score, hidden_state

In [108]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

For english

In [109]:
# f = open('en.json',)
# data = json.load(f)

# idt_rationale_list = []
# idt_token_list = []

# rationale_list = []
# token_list = []

# for i in data:
#     total = []
#     for annot in data[i]['annotators']:
#         total += annot['target']
#     if 'None' in total: # at least one people think it is not hate speech, only keep all 3 think it is hateful
#         pass
#     else:
#         if len([each for each in idt_list if each.lower() in data[i]['post_tokens']])>0: # hate speech with idt           
#             idt_token_list.append(' '.join(data[i]['post_tokens']))
#             idt_rationale_list.append(data[i]['rationales'])
#         else:
#             token_list.append(' '.join(data[i]['post_tokens']))
#             rationale_list.append(data[i]['rationales'])
# f.close()

convert to token id and start measure

In [110]:
# #For Germany
# df = pd.read_table('de.txt', sep='	', lineterminator='\n', header=None)
# df.columns = ['text', 'binary', 'multi'] 
# df = df.loc[df['binary']=='OFFENSE']
# df